In [13]:
import asyncio
import panel as pn
pn.extension()

C:\Users\admin\AppData\Local\Temp\ipykernel_39560\3887293949.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


## Query endpoint

Při získávání dat z graphql koncového bodu (endpointu), je potřeba projít autentizací. Typicky k tomuto slouží tokeny. Token je následně použit pro identifikaci uživatele a s ním je hodnocena autorizace pro čtení jednotlivých datových struktur (včetně atributů).

V našem případě je k dispozici možnost autentizace s heslem a jménem, kdy získáme požadovaný token. Ten je následně zařazován do požadavků jako cookie.
Funkce `getToken` vrací token na základě `username` a `password`. 
Funkce `query` "zabaluje" dotaz a token a vytváří funkci, která potřebuje již jen parametry dotazu a vrací celý paket s odpovědí.

In [2]:
import aiohttp
async def getToken(username, password):
    keyurl = "http://host.docker.internal:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)
            
def query(q, token):
    async def post(variables):
        gqlurl = "http://host.docker.internal:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post 

## Flatten data

Pojem flatten je spojen s převodem dat se stromovou strukturou, tedy obsahujících více seznamů (lists) a slovníků (dictionaries) do formy homogenních dat.
Kód níže, specificky funkce `flatten` vytváří homogenní strukturu (viz tabulka v Excelu, která slouží jako vstupní data pro kontingenční tabulku).

In [3]:
from itertools import product
from functools import reduce

def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            # print("flatList", row)
            yield row
            
def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    # print("flatDict.result", result)
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced
        
def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

## Prezentace dat

Převod dat na strukturu


In [12]:
import pandas as pd
def toTable(flatData):
    return pd.DataFrame(flatData)

## Finální kompozice 

Spojíme dohromady:
- čtení dat
- konverzi dat
- prezentaci dat

In [19]:
      
username = "john.newbie@world.com"
password = "john.newbie@world.com"

queryStr = """
{
  result: userPage(
    where: {memberships: {group: {name: {_ilike: "%uni%"}}}}
  ) {
    id
    email
    name
    surname
    
    presences {
      event {
        id
        name
        startdate
        enddate
        eventType {
          id
          name
        }
      }
      presenceType {
        id
        name
      }
    }
  }
}
"""

mappers = {
    "id": "id",
    "name": "name",
    "surname": "surname",
    "email": "email",
    "eventid": "presences.event.id",
    "eventname": "presences.event.name",
    # "eventduration": "presences.event.duration",
    "startdate": "presences.event.startdate",
    "enddate": "presences.event.enddate",
    "eventTypeid": "presences.event.eventType.id",
    "eventTypename": "presences.event.eventType.name",
    "presenceTypeid": "presences.presenceType.id",
    "presenceTypename": "presences.presenceType.name"
}

async def fullPipe():
    global pandasData
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("result", None)

    flatData = flatten(result, {}, mappers)
    pandasData = toTable(flatData)
    return pandasData

fullPipe()
# print(list(flatData))

,id,name,surname,email,eventid,eventname,startdate,enddate,eventTypeid,eventTypename,presenceTypeid,presenceTypename
0,89d1e724-ae0f-11ed-9bd8-0242ac110002,Jana,Newbie,jana.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
1,89d1f34a-ae0f-11ed-9bd8-0242ac110002,Jolana,Newbie,jolana.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
2,89d1f3cc-ae0f-11ed-9bd8-0242ac110002,Jitka,Newbie,jitka.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
3,89d1f430-ae0f-11ed-9bd8-0242ac110002,Jaroslava,Newbie,jaroslava.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
4,89d1f48a-ae0f-11ed-9bd8-0242ac110002,Lada,Newbie,lada.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
5,89d1f4e4-ae0f-11ed-9bd8-0242ac110002,Ludmila,Newbie,ludmila.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
6,89d1f534-ae0f-11ed-9bd8-0242ac110002,Lucie,Newbie,lucie.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
7,89d1f58e-ae0f-11ed-9bd8-0242ac110002,Nola,Newbie,nola.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
8,89d1f5de-ae0f-11ed-9bd8-0242ac110002,Neva,Newbie,neva.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
9,89d1f638-ae0f-11ed-9bd8-0242ac110002,Nora,Newbie,nora.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen


## Prezentace dat

Pomocí knihovny panel provedem prezentaci

In [ ]:
pn.panel(fullPipe()).servable()
# pn.panel(pandasData).servable()


# data = [{"a": 'value a', "b": 'value b'}, {"a": 'value a 1', "b": 'value b 1'}]
# df = pd.DataFrame(data)
# pn.panel(df).servable()